In [4]:
class ASTNode:
    def __init__(self, node_type, value=None, left=None, right=None):
        self.node_type = node_type  # 'operator' or 'operand'
        self.value = value          # e.g., 'AND', 'OR', or 'age > 30'
        self.left = left            # left child node
        self.right = right          # right child node


In [5]:
import re

def create_rule(rule_string):
    # Split rule string by operators and parentheses while keeping them
    tokens = re.split(r'(\(|\)|AND|OR)', rule_string.replace(' ', ''))
    stack = []
    operators = []
    
    for token in tokens:
        if token == '':
            continue
        if token == '(':
            operators.append(token)  # Push '(' onto the operator stack
        elif token == ')':
            # Process all operators until '(' is found
            while operators and operators[-1] != '(':
                process_operator(stack, operators.pop())
            operators.pop()  # Remove '(' from the stack
        elif token in ('AND', 'OR'):
            # Process the operator with precedence handling
            while operators and precedence(operators[-1]) >= precedence(token):
                process_operator(stack, operators.pop())
            operators.append(token)
        elif re.match(r'[a-zA-Z]+(>|<|=)[0-9a-zA-Z\'"]+', token):
            # Push operand to the stack (conditions like 'age > 30')
            node = ASTNode(node_type='operand', value=token)
            stack.append(node)

    # Process remaining operators
    while operators:
        process_operator(stack, operators.pop())
    
    return stack.pop() if stack else None

def process_operator(stack, operator):
    # Ensure we have two operands to apply the operator to
    if len(stack) < 2:
        raise ValueError(f"Not enough operands for operator {operator}")
    right = stack.pop()
    left = stack.pop()
    operator_node = ASTNode(node_type='operator', value=operator, left=left, right=right)
    stack.append(operator_node)

def precedence(operator):
    # Define operator precedence
    if operator == 'AND':
        return 2
    elif operator == 'OR':
        return 1
    return 0


In [6]:
def combine_rules(rules):
    if not rules:
        return None
    combined_root = rules[0]
    
    for i in range(1, len(rules)):
        combined_node = ASTNode(node_type='operator', value='AND')
        combined_node.left = combined_root
        combined_node.right = rules[i]
        combined_root = combined_node
        
    return combined_root


In [7]:
def evaluate_rule(ast_node, data):
    if ast_node.node_type == 'operator':
        print(f"Evaluating operator: {ast_node.value}")
        left_result = evaluate_rule(ast_node.left, data)
        right_result = evaluate_rule(ast_node.right, data)
        print(f"Left result: {left_result}, Right result: {right_result}")
        
        if ast_node.value == 'AND':
            return left_result and right_result
        elif ast_node.value == 'OR':
            return left_result or right_result
    
    elif ast_node.node_type == 'operand':
        field, operator, threshold = re.split(r'(>|<|=)', ast_node.value)
        field = field.strip()
        threshold = threshold.strip().strip("'\"")
        field_value = data.get(field)
        
        print(f"Evaluating operand: {field} {operator} {threshold} with value {field_value}")
        
        if field_value is None:
            raise ValueError(f"Field {field} is missing in the input data")
        
        if isinstance(field_value, (int, float)):
            threshold = float(threshold)
            
        if operator == '>':
            return field_value > threshold
        elif operator == '<':
            return field_value < threshold
        elif operator == '=':
            return str(field_value) == str(threshold)
    
    return False


In [8]:
# Sample rules
rule1 = "((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5)"
rule2 = "((age > 30 AND department = 'Marketing')) AND (salary > 20000 OR experience > 5)"

# Convert them into AST
ast_rule1 = create_rule(rule1)
ast_rule2 = create_rule(rule2)

# Combine both rules using AND
combined_ast = combine_rules([ast_rule1, ast_rule2])

# Sample data to evaluate the rule
data1 = {"age": 35, "department": "Sales", "salary": 60000, "experience": 3}
data2 = {"age": 22, "department": "Marketing", "salary": 30000, "experience": 2}

# Evaluate the combined rule for different data
result1 = evaluate_rule(combined_ast, data1)  # Expected: True
result2 = evaluate_rule(combined_ast, data2)  # Expected: False

print(f"Result for data1: {result1}")
print(f"Result for data2: {result2}")


Evaluating operator: AND
Evaluating operator: AND
Evaluating operator: OR
Evaluating operator: AND
Evaluating operand: age > 30 with value 35
Evaluating operand: department = Sales with value Sales
Left result: True, Right result: True
Evaluating operator: AND
Evaluating operand: age < 25 with value 35
Evaluating operand: department = Marketing with value Sales
Left result: False, Right result: False
Left result: True, Right result: False
Evaluating operator: OR
Evaluating operand: salary > 50000 with value 60000
Evaluating operand: experience > 5 with value 3
Left result: True, Right result: False
Left result: True, Right result: True
Evaluating operator: AND
Evaluating operator: AND
Evaluating operand: age > 30 with value 35
Evaluating operand: department = Marketing with value Sales
Left result: True, Right result: False
Evaluating operator: OR
Evaluating operand: salary > 20000 with value 60000
Evaluating operand: experience > 5 with value 3
Left result: True, Right result: False
L